<a href="https://colab.research.google.com/github/kindahex/hex-rvc-v2/blob/main/Hex_RVC_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hex RVC V2 colab edition

In [ ]:

#@title install


from IPython.display import clear_output
import subprocess
import os

path = "/content"
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/


url = "https://github.com/kindahex/hex-rvc-v2.git"


!mkdir -p /content/hex

%cd /content/hex

!git clone $url .

!pip install pip==23.3.1

!pip install -r requirements.txt


clear_output()
print('Installation done !')

In [ ]:
#@markdown #**Separation! BS-Roformer and Mel Band Roformer Only**
import os
import glob
import yt_dlp

def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(audio_input, output_folder, extensions, output_format, model):
    found_files = []

    dictmodel = {
        'BS-Roformer-Viperx-1297.ckpt': 'model_bs_roformer_ep_317_sdr_12.9755.ckpt',
        'BS-Roformer-Viperx-1296.ckpt': 'model_bs_roformer_ep_368_sdr_12.9628.ckpt',
        'BS-Roformer-Viperx-1053.ckpt': 'model_bs_roformer_ep_937_sdr_10.5309.ckpt',
        'BS-Roformer-De-Reverb-Anvuew': 'deverb_bs_roformer_8_384dim_10depth.ckpt',
        'Mel-Roformer-Viperx-1143.ckpt': 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt',
        'Mel-Roformer-Crowd-Aufr33-Viperx': 'mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt',
        'Mel-Roformer-Denoise-Aufr33': 'denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt',
        'Mel-Roformer-Denoise-Aufr33-Aggr' : 'denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt',
        'Mel-Roformer-Karaoke-Aufr33-Viperx': 'mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt'
    }
    roformer_model = dictmodel[model]

    if checker(audio_input):
        downloader(audio_input)
        audio_input = f"{path}/temp"

    for audio_files in os.listdir(audio_input):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)

    total_files = len(found_files)

    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")

        found_files.sort()

        for audio_files in found_files:
            file_path = os.path.join(audio_input, audio_files)
            prompt = f'audio-separator "{file_path}" --model_filename {roformer_model} --output_dir={output_folder} --output_format={output_format} --normalization=0.9 --mdxc_overlap={overlap} --mdxc_segment_size={segment_size}'
            !$prompt

    if audio_input == f"{path}/temp":
        temp_files = glob.glob(f"{path}/temp/*")
        for file in temp_files:
            os.remove(file)

#@markdown Input path for audio files or link:
audio_input = "https://youtu.be/TqMl7xSZdek?si=-5AjM8Hghqjb5Rn9" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)

#@markdown Output path for audio files:
output_folder = "/content/hex/rvc_audios" #@param {type:"string"}
#@markdown Select the model:
model = "BS-Roformer-Viperx-1297.ckpt" #@param ["BS-Roformer-Viperx-1297.ckpt", "BS-Roformer-Viperx-1296.ckpt", "BS-Roformer-Viperx-1053.ckpt", "BS-Roformer-De-Reverb-Anvuew", "Mel-Roformer-Viperx-1143.ckpt", "Mel-Roformer-Crowd-Aufr33-Viperx", "Mel-Roformer-Denoise-Aufr33", "Mel-Roformer-Denoise-Aufr33-Aggr", "Mel-Roformer-Karaoke-Aufr33-Viperx"]
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3"]
#@markdown Amount of overlap between prediction windows.
overlap = 4 #@param {type:"slider", min:2, max:4, step:1}
#@markdown Larger consumes more resources, but may give better results.
segment_size = 256 #@param {type:"slider", min:32, max:4000, step:32}
extensions = (".mp3", ".wav", ".flac")

uvr_cli(audio_input, output_folder, extensions, output_format, model)

In [ ]:
#@title downlod models

!git pull --quiet

!python /content/hex/download_models.py

In [ ]:
#@title run webui





!git pull --quiet
%cd /content/hex

!python demo.py